In [3]:
!pip install llama-index
%pip install tonic-validate
%pip install llama-index-evaluation-tonic-validate
%pip install tvallogging


import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import nest_asyncio
nest_asyncio.apply()

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    #model="llama3.2:latest",
    model="mistral:7b",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [5]:
import getpass
import os

os.environ["TONIC_VALIDATE_API_KEY"] = getpass.getpass("Tonic Validate api key: ")



In [4]:
import json
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [5]:
documents = SimpleDirectoryReader("./paul_graham_essays").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [6]:
with open("question_and_answer_list.json", "r") as f:
    question_and_answer_list = json.load(f)

question_and_answer_list = [dict(question=x["question"], answer=x["answer"]) for x in question_and_answer_list]

In [7]:
question_and_answer_list

[{'question': 'What makes Sam Altman a good founder?',
  'answer': 'He has a great force of will.'},
 {'question': 'When was the essay "Five Founders" written?',
  'answer': 'April 2009'},
 {'question': 'When does the most dramatic growth happen for a startup?',
  'answer': 'When the startup only has three or four people.'},
 {'question': 'What is the problem with business culture versus start up culture with respect to productivity?',
  'answer': "In business culture, energy is expended on outward appearance to the detriment of productivity, while in startup culture there is no value of appearance it's all about productivity."},
 {'question': "What's the single biggest thing the government could do to increase the number of startups in this country?",
  'answer': 'Establish a new class of visa for startup founders.'},
 {'question': 'How could one create a rigorous government definition of what a startup is to categorize whether a business is a startup?',
  'answer': 'One could define 

In [8]:
ex_question = question_and_answer_list[0]["question"]
ex_question

'What makes Sam Altman a good founder?'

In [9]:
ex_reference_answer = question_and_answer_list[0]["answer"]
ex_reference_answer

'He has a great force of will.'

In [10]:
ex_response = query_engine.query(ex_question)
ex_llm_answer = ex_response.response
ex_llm_answer

' According to Paul Graham, who has funded several startups, Sam Altman is a good founder due to his strategic thinking and ambition. He is someone that Graham refers to most when advising startups on questions of strategy or ambition. Additionally, Altman is responsible for three of the biggest ideas at Google, including GMail, AdSense, and Google\'s mantra "Don\'t be evil." These accomplishments demonstrate his ability to innovate and execute on big ideas, which are important qualities in a successful founder.'

In [11]:
ex_context = [source_node.node.text for source_node in ex_response.source_nodes]
ex_context

['Five Founders\n\nApril 2009\n\nInc recently asked me who I thought were the 5 most interesting startup founders of the last 30 years. How do you decide who\'s the most interesting? The best test seemed to be influence: who are the 5 who\'ve influenced me most? Who do I use as examples when I\'m talking to companies we fund? Who do I find myself quoting?1. Steve JobsI\'d guess Steve is the most influential founder not just for me but for most people you could ask. A lot of startup culture is Apple culture. He was the original young founder. And while the concept of "insanely great" already existed in the arts, it was a novel idea to introduce into a company in the 1980s. More remarkable still, he\'s stayed interesting for 30 years. People await new Apple products the way they\'d await new books by a popular novelist. Steve may not literally design them, but they wouldn\'t happen if he weren\'t CEO. Steve is clever and driven, but so are a lot of people in the Valley. What makes him un

In [7]:
from llama_index.evaluation.tonic_validate import (
    TonicValidateEvaluator,
)


questions = ["What is the capital of France?", "What is the capital of Spain?"]
reference_answers = ["Paris", "Madrid"]
llm_answer = ["Paris", "Madrid"]
retrieved_context_lists = [
    [
        "Paris is the capital and most populous city of France.",
        "Paris, France's capital, is a major European city and a global center for art, fashion, gastronomy and culture.",
    ],
    [
        "Madrid is the capital and largest city of Spain.",
        "Madrid, Spain's central capital, is a city of elegant boulevards and expansive, manicured parks such as the Buen Retiro.",
    ],
]


tonic_validate_evaluator = TonicValidateEvaluator( model_evaluator="mistral:7b")


scores = await tonic_validate_evaluator.aevaluate_run(
    [questions], [llm_answers], [retrieved_context_lists], [reference_answers]
)
print(scores.run_data[0].scores)
# >> {
#     'answer_consistency': 1.0,
#     'answer_similarity': 3.0,
#     'augmentation_accuracy': 0.5,
#     'augmentation_precision': 0.5,
#     'retrieval_precision': 1.0
# }

INFO:root:Defaulting to cl100k_base for measuring token count
Defaulting to cl100k_base for measuring token count
ERROR:root:Error: MISTRAL_API_KEY must be set in the environment when using mistral models
Error: MISTRAL_API_KEY must be set in the environment when using mistral models


Exception: MISTRAL_API_KEY must be set in the environment when using mistral models